In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.offline as pyo

pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

# 1950~2067년
dfDeathWoman = pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
dfDeathMan = pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
npDeathWoman = np.array(dfDeathWoman)
npDeathMan = np.array(dfDeathMan)

# 1950~2047년
dfDoctorData = pd.read_excel('../../data/doctorData.xlsx').set_index(['년도']).iloc[:98,:]
npPassDoctor = np.around(np.array(dfDoctorData[['의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]))
npDoctorExam = np.around(np.array(dfDoctorData[['의사고시 합격자수','의사고시 불합격자수']]))
npPopulation = np.array(dfDoctorData['추계인구'])
npPassDoctorRate = np.array(dfDoctorData[['의대졸비율/남', '의대졸비율/여', '의전졸비율/남', '의전졸비율/여', '불합비율/남', '불합비율/여']])

# 2011~2016년
npDoctorAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))


dfFirstDoctor = pd.read_excel('../../data/firstDoctor.xlsx')
npFirstDoctor = np.array(dfFirstDoctor)[:,1:-1]

# 1950~2020년 연령분포
dfKoreaAgePopRateData = pd.read_excel('../../data/koreaAgePopRateData.xlsx')
npKoreaAgePopRateDataMan = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='MALE'])[:,2:]
npKoreaAgePopRateDataWoman = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='FEMALE'])[:,2:]

In [2]:
def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

def makeAlivePerson(npData,year):
    
    deathRate = np.array([npDeathMan[year], npDeathWoman[year]])
    deadPerson = np.around((npData*deathRate))
    resultData = npData - deadPerson
    
    return [resultData, deadPerson]

def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    retirePerson = np.around(npData*valueList)
    result = npData - retirePerson
    
    return [result, retirePerson]

def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

def makeNewPerson(npData, tuningSet):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열 98 x 6 x 100
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열 98 x 2 x 100
    newPersonArray = np.zeros((yearSize,2,oldSize))

    modelAry = makeArrayUseModel(tuningSet)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    
    return resultData

def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray 

def makeResultPersonArray(newPersonArray, tuningSet):
    #누적인원 배열
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    
    # 98 x 2 x 100 
    resultPersonArray = np.zeros(sizeArray)
    deadPersonArray = np.zeros(sizeArray)
    retirePersonArray = np.zeros(sizeArray)
    
    resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
    resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))
    
    
    
    
    for i in range(2,yearSize):
        # 1살 올리기 / shiftData -> 2 x 100
        shiftData = shiftOld(resultPersonArray[i-1])
        # 사망률 적용 / aliveData -> 2 x 100
        aliveData = makeAlivePerson(shiftData, i)
        deadPersonArray[i] = aliveData[1]
        # 은퇴율 적용 / workData -> 2 x 100
        workData = makeWorkPerson(aliveData[0], tuningSet)
        retirePersonArray[i] = workData[1]
        # 최종 계산
        resultPersonArray[i] =  workData[0] + newPersonArray[i]

            
            
    return [resultPersonArray,newPersonArray,deadPersonArray,retirePersonArray]


def sumPeopleUseAge(npData):
    yearSize = len(npData)
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData

def calculateCost(npData):   
    resultData = np.zeros([6,5,5])

    for i in range(6):
        manNumCost = npData[66-i][0][1:6] - npDoctorAgeData[5-i][8:13]
        womanNumCost = npData[66-i][1][1:6] - npDoctorAgeData[5-i][14:19]
        totalNumCost = npData[66-i][2][1:6] - npDoctorAgeData[5-i][2:7]

        manRateCost = npData[66-i][0][7:12] - npDoctorAgeData[5-i][26:31]
        womanRateCost = npData[66-i][1][7:12] - npDoctorAgeData[5-i][32:37]

        npCostArray = np.array([manNumCost, womanNumCost, totalNumCost, 
                    manRateCost, womanRateCost])

        resultData[i] = npCostArray

    return resultData

def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

def makeDataFrame(npData):
    dfList = []
    startYear = 1950
    for i in range(4):
        for j in range(2):
            dfResult = pd.DataFrame(npData[i][0][j]).T
            indexList = []
            indexList.append(startYear)

            for k in range(1,98):
                dfResult = pd.concat([dfResult,pd.DataFrame(npData[i][k][j]).T])
                indexList.append(k+startYear)

            dfResult.index = indexList 
            dfList.append(dfResult)
            
    return dfList

def makeThousandPerDoctor(dfResultPerson, npPopulation):
    startYear = 1950
    dfResultPersonSum = dfResultPerson[2].sum(axis=1)
    npResultPersonSum = np.array(dfResultPersonSum)
    dfThousandPerDoctor = pd.DataFrame(npResultPersonSum/npPopulation*1000)
    dfThousandPerDoctor.index = range(startYear, 2048)
    return dfThousandPerDoctor

def makeFuturePerson(npBasicPopulation):
    yearSize = 98
    optSize = len(npBasicPopulation)
    npFuturePerson = np.zeros([yearSize,6])
    
    for i in range(optSize):
        npDoctorExam[i+76][0] = (npBasicPopulation[i][0]*0.94)+(npBasicPopulation[i][1]*0.94) #의대 합격자
        npDoctorExam[i+76][1] = (npBasicPopulation[i][0]*0.06)+(npBasicPopulation[i][1]*0.06) # 의대 불합격자 
        
        popSum = npBasicPopulation[i][0]+npBasicPopulation[i][1]+npDoctorExam[i+75][1]

        npPassDoctorRate[i+76][0] = (npBasicPopulation[i][0]*npBasicPopulation[i][2])/popSum #의대남자비율
        npPassDoctorRate[i+76][1] = (npBasicPopulation[i][0]*(1-npBasicPopulation[i][2]))/popSum #의대여자비율
        npPassDoctorRate[i+76][2] = (npBasicPopulation[i][1]*npBasicPopulation[i][3])/popSum #의전원남자비율
        npPassDoctorRate[i+76][3] = (npBasicPopulation[i][1]*(1-npBasicPopulation[i][3]))/popSum #의전원여자비율
        npPassDoctorRate[i+76][4] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][0]+npPassDoctorRate[i+76][2])/popSum # 불합격남자비율
        npPassDoctorRate[i+76][5] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][1]+npPassDoctorRate[i+76][3])/popSum # 불합격여자비율
        
        npFuturePerson[i+76][0] = np.around(popSum * npPassDoctorRate[i+76][0])
        npFuturePerson[i+76][1] = np.around(popSum * npPassDoctorRate[i+76][1])
        npFuturePerson[i+76][2] = np.around(popSum * npPassDoctorRate[i+76][2])
        npFuturePerson[i+76][3] = np.around(popSum * npPassDoctorRate[i+76][3])
        npFuturePerson[i+76][4] = np.around(popSum * npPassDoctorRate[i+76][4])
        npFuturePerson[i+76][5] = np.around(popSum * npPassDoctorRate[i+76][5])
        
    return npFuturePerson





In [3]:
def makeDoctorData(npData, tuningSet):
    npFuturePerson = makeFuturePerson(npData)
    npNewPassDoctor = npPassDoctor + npFuturePerson
    newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])
    resultData = makeResultPersonArray(newPerson,tuningSet[1])

    dfData = makeDataFrame(resultData)

    dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
    dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
    dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
    dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
    dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, npPopulation) # 1000명당 의사수
    
    return [dfResultPerson, dfNewPerson, dfDeadPerson, dfRetirePerson, dfThousandPerDoctor]

In [4]:
#의대수, 의전원수, 의대남비율, 의전원남비율 
npBasicPopulation = np.zeros([22,4])
for i in range(22):
    #의대 합격자 , 의전원 합격자 , 의대합격자성비(남자), 의전원합격자성비(남자)
    npBasicPopulation[i] = np.array([3400,50,0.6,0.6])
    
tuningSetAgeRate = [[0.9, 0.7, 0.8, 0.8, 0.7, 0.7],[26,26,28,28,27,27],[40, 40, 40, 40, 40, 40]]# a값, 시작값(x의 최소값), x의 최대값(0으로수렴시작하는 값)
tuningSetRetireRate = [[1.13, 1.07],[30, 30],[2.4, 1.6]] # a값, y값이 0보다 커지기 시작하는 부분, 은퇴율의 가중치(값의 정확성을 위해)
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]

npFuturePerson = makeFuturePerson(npBasicPopulation) # 향후 신규의사
npNewPassDoctor = npPassDoctor + npFuturePerson # 기존 의사 + 향후신규의사
newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])#위 데이터를 가지고 모델써서 신규의사를 분배 (여기서 첫번째 튜닝, tuningSetAgeRate)
resultData = makeResultPersonArray(newPerson,tuningSet[1]) # 최종 의사 수. 사망, 은퇴 등을 처리해 

dfData = makeDataFrame(resultData) #리스트형태를 데이터 프레임으로 만듦, 

dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, npPopulation) # 1000명당 의사수


In [5]:
dfDeadPerson

[       0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
 1950  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1951  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1952  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1953  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1954  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1955  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1956  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1957  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1958  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1959  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1960  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 1961  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [17]:
dfDeadPerson[0].iloc[:,26:]

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
1950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1952,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1953,0.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1954,0.0,0.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1955,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1956,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1957,0.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1958,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1959,0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def getDataframe(dfResultPerson):
    #남성 의사 인력 DataFrame, 변수명 manDoc
    manDoc = dfResultPerson[0].iloc[:,25:]
    index = list(range(1950,2048))
    manDoc = manDoc.set_index([index])
    
    #여성 의사 인력 DataFrame, 변수명 womDoc
    womDoc = dfResultPerson[1].iloc[:,25:]
    index = list(range(1950,2048))
    womDoc = womDoc.set_index([index])
    
    bothdoc = dfResultPerson[2]
    index = list(range(1950,2048))
    bothdoc = bothdoc.set_index([index])
    return [manDoc, womDoc, bothdoc]

def slicingPerson(year):
    value = []
    docData =getDataframe(dfResultPerson)
    value.append(docData[0].iloc[year])
    value.append(docData[1].iloc[year])
    value.append(docData[2].iloc[year])
    return value



def makeFigureManWom(year):
    yearValue = int(year)
    yearIndex = yearValue-1950

    womOfYear = pd.DataFrame(slicingPerson(yearIndex)[1])
    womOfYear = womOfYear.rename_axis('age').reset_index()
    numOfWom = womOfYear[yearValue].tolist()

    docManYear = pd.DataFrame(slicingPerson(yearIndex)[0])
    docManYear = docManYear.rename_axis('age').reset_index()
    numOfMan = docManYear[yearValue].tolist()

    
    trace3 = go.Bar(name='Man', x=list(docManYear['age']), y=list(docManYear[yearValue]))
    trace4 = go.Bar(name = 'Woman', x=list(womOfYear['age']), y=list(womOfYear[yearValue]))

    data = [trace3, trace4]
    layout = go.Layout(title=str(year)+'년 연령별 의사 수')
    fig = go.Figure(data=data, layout=layout)

    fig.update_layout(
        height=500,
        width=1000,
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=0),
        showlegend=False,
    )
    return fig




In [22]:
makeFigureManWom(2010)

In [8]:
def thousandDocGet(dfThousandPerDoctor):
    docPer1000 = dfThousandPerDoctor
    docPer1000 = docPer1000.dropna(axis = 0, how = 'any')
    #index 변경
    index = list(range(1950,2048))
    docPer1000 = docPer1000.set_index([index])
    docPer1000.columns = ['docnum']
    return docPer1000

def OECDDocGet():
    OECDPer1000 = pd.read_excel('../../data/OECD의사수.xlsx')
    OECDPer1000 = OECDPer1000.groupby(['구분','년도']).mean()
    OECDPer1000 = OECDPer1000.iloc[0:60,:]
    OECDPer1000 = OECDPer1000.reset_index()
    return OECDPer1000

In [9]:
def makeFigureDocPer1000():
    fig = go.Figure()
    docPer1000 = thousandDocGet(dfThousandPerDoctor)
    years = list(docPer1000.index)  #연도
    docPerThousend = list(docPer1000['docnum'])  # 1000명당 국내 의사 수 
    OECDPer1000 = OECDDocGet()
    year=list(OECDPer1000['년도'])
    OECDValue = list(OECDPer1000['값'])
    
    fig.add_trace(go.Scatter(x=years, y=docPerThousend,
                        mode='lines',
                        name='KOREA'))
    fig.add_trace(go.Scatter(x=year, y=OECDValue,
                        mode='lines',
                        name='OECD'))

    return fig

In [14]:
def makeFigureSumDoc():
    fig = go.Figure()
    
    year = list(range(1950, 2048))
    docData =getDataframe(dfResultPerson)
    manSumDoc = list(docData[0].sum(axis=1))
    womSumDoc = list(docData[1].sum(axis=1))
    bothSumDoc = list(docData[2].sum(axis=1))
    
    fig.add_trace(go.Scatter(x=year, y=manSumDoc,
                        mode='lines',
                        name='MAN'))
    fig.add_trace(go.Scatter(x=year, y=womSumDoc,
                        mode='lines',
                        name='WOMAN'))
    fig.add_trace(go.Scatter(x=year, y=bothSumDoc,
                        mode='lines',
                        name='BOTH'))

    return fig
makeFigureSumDoc()

# 은퇴 연도별 그래프

In [11]:
def getDataframe2(dfRetirePerson):
    #남성 의사 인력 DataFrame, 변수명 manDoc
    manDoc = dfRetirePerson[0]
    index = list(range(1950,2048))
    manDoc = manDoc.set_index([index])
    
    #여성 의사 인력 DataFrame, 변수명 womDoc
    womDoc = dfRetirePerson[1]
    index = list(range(1950,2048))
    womDoc = womDoc.set_index([index])
    
    bothdoc = dfRetirePerson[2]
    index = list(range(1950,2048))
    bothdoc = bothdoc.set_index([index])
    return [manDoc, womDoc, bothdoc]

def makeFigureRetireDoc():
    fig = go.Figure()
    
    year = list(range(1950, 2048))
    docData =getDataframe2(dfRetirePerson)
    manSumDoc = list(docData[0].sum(axis=1))
    womSumDoc = list(docData[1].sum(axis=1))
    bothSumDoc = list(docData[2].sum(axis=1))
    
    fig.add_trace(go.Scatter(x=year, y=manSumDoc,
                        mode='lines',
                        name='MAN'))
    fig.add_trace(go.Scatter(x=year, y=womSumDoc,
                        mode='lines',
                        name='WOMAN'))
    fig.add_trace(go.Scatter(x=year, y=bothSumDoc,
                        mode='lines',
                        name='BOTH'))

    return fig
makeFigureRetireDoc()

# 사망 연도별 그래프

In [12]:
def getDataframe3(dfDeadPerson):
    #남성 의사 인력 DataFrame, 변수명 manDoc
    manDoc = dfDeadPerson[0]
    index = list(range(1950,2048))
    manDoc = manDoc.set_index([index])
    
    #여성 의사 인력 DataFrame, 변수명 womDoc
    womDoc = dfDeadPerson[1]
    index = list(range(1950,2048))
    womDoc = womDoc.set_index([index])
    
    bothdoc = dfDeadPerson[2]
    index = list(range(1950,2048))
    bothdoc = bothdoc.set_index([index])
    return [manDoc, womDoc, bothdoc]

def makeFigureDeadDoc():
    fig = go.Figure()
    
    year = list(range(1950, 2048))
    docData =getDataframe3(dfDeadPerson)
    manSumDoc = list(docData[0].sum(axis=1))
    womSumDoc = list(docData[1].sum(axis=1))
    bothSumDoc = list(docData[2].sum(axis=1))
    
    fig.add_trace(go.Scatter(x=year, y=manSumDoc,
                        mode='lines',
                        name='MAN'))
    fig.add_trace(go.Scatter(x=year, y=womSumDoc,
                        mode='lines',
                        name='WOMAN'))
    fig.add_trace(go.Scatter(x=year, y=bothSumDoc,
                        mode='lines',
                        name='BOTH'))

    return fig
makeFigureDeadDoc()

# 신규 의사 연도별 그래프 

In [13]:
def getDataframe4(dfNewPerson):
    #남성 의사 인력 DataFrame, 변수명 manDoc
    manDoc = dfNewPerson[0]
    index = list(range(1950,2048))
    manDoc = manDoc.set_index([index])
    
    #여성 의사 인력 DataFrame, 변수명 womDoc
    womDoc = dfNewPerson[1]
    index = list(range(1950,2048))
    womDoc = womDoc.set_index([index])
    
    bothdoc = dfNewPerson[2]
    index = list(range(1950,2048))
    bothdoc = bothdoc.set_index([index])
    return [manDoc, womDoc, bothdoc]

def makeFigureNewDoc():
    fig = go.Figure()
    
    year = list(range(1950, 2048))
    docData =getDataframe4(dfNewPerson)
    manSumDoc = list(docData[0].sum(axis=1))
    womSumDoc = list(docData[1].sum(axis=1))
    bothSumDoc = list(docData[2].sum(axis=1))
    
    fig.add_trace(go.Scatter(x=year, y=manSumDoc,
                        mode='lines',
                        name='MAN'))
    fig.add_trace(go.Scatter(x=year, y=womSumDoc,
                        mode='lines',
                        name='WOMAN'))
    fig.add_trace(go.Scatter(x=year, y=bothSumDoc,
                        mode='lines',
                        name='BOTH'))

    return fig
makeFigureNewDoc()

In [32]:
def makeFigureNewDoc():
    fig = go.Figure()
    
    fig = go.Figure()

    valueList=np.zeros(100)

    a = 1.1
    b = 30
    c = 2

    for i in range(100):
        result = ((a**(i-b))-1)*c/((a**(100-b))-1)
        valueList[i] = 0 if result<0 else 1 if result>1 else  result

    dfValue1 = list(valueList)
    dfValue2 = list(range(100))
    fig.add_trace(go.Scatter(x=dfValue2, y=dfValue1,mode='lines', name='MAN'))
    
    fig.update_layout(
        height=500,
        width=1000,
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=0),
        showlegend=False,
    )

    return fig

makeFigureNewDoc()